### **Práctica: Generación de Rostros con VAE usando CelebA**  
**Objetivo**: Entrenar un Variational Autoencoder (VAE) convolucional para generar rostros nuevos a partir del dataset CelebA.  

---

### **1. Configuración Inicial**  
**Objetivo**: Preparar el entorno y el dataset.  

**Instrucciones**:  

1. **Descargar CelebA**:  
   - El dataset contiene ~200k imágenes de rostros.  
   - Utilizar `kagglehub` para descargar el dataset directamente desde Kaggle.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/celeba-dataset")

print("Path to dataset files:", path)

2. **Preprocesamiento**:  
   - Redimensionar imágenes a 64x64 píxeles (si no están ya en ese tamaño).  
   - Normalizar píxeles al rango `[0, 1]`.  

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image

# Path to the dataset
img_dir = os.path.join(path, 'img_align_celeba/img_align_celeba')

# Check if the directory exists
if not os.path.exists(img_dir):
    img_dir = os.path.join(path, 'img_align_celeba', 'img_align_celeba')
    if not os.path.exists(img_dir):
        print(f"Could not find the images directory. Available files in {path}:")
        print(os.listdir(path))

# List all image files
image_files = [f for f in os.listdir(img_dir) if f.endswith('.jpg') or f.endswith('.png')]
print(f"Total number of images found: {len(image_files)}")

# Function to preprocess images
def preprocess_image(img_path, target_size=(64, 64)):
    # Load and resize image
    img = Image.open(img_path)
    img = img.resize(target_size, Image.LANCZOS)
    
    # Convert to numpy array and normalize to [0, 1]
    img_array = np.array(img).astype('float32') / 255.0
    
    return img_array

# Create a sample of images to process (limit to 5000 images to save memory)
sample_size = min(5000, len(image_files))
sample_files = image_files[:sample_size]

# Process the sampled images
processed_images = []

# Simple progress tracking
start_time = time.time()
update_interval = max(1, sample_size // 20)  # Update every 5% of progress
print(f"Processing {sample_size} images...")

for i, img_file in enumerate(sample_files):
    img_path = os.path.join(img_dir, img_file)
    processed_img = preprocess_image(img_path)
    processed_images.append(processed_img)
    
    # Print progress update
    if (i + 1) % update_interval == 0 or i == sample_size - 1:
        progress = (i + 1) / sample_size * 100
        elapsed = time.time() - start_time
        remaining = elapsed / (i + 1) * (sample_size - i - 1) if i > 0 else 0
        print(f"Progress: {progress:.1f}% ({i+1}/{sample_size}) - Elapsed: {elapsed:.1f}s - Est. remaining: {remaining:.1f}s")

# Convert to numpy array
processed_images = np.array(processed_images)
print(f"Processing completed in {time.time() - start_time:.1f} seconds")
print(f"Processed images shape: {processed_images.shape}")
print(f"Min value: {processed_images.min()}, Max value: {processed_images.max()}")

3. **Visualizar algunas imágenes procesadas**:  
   - Verifiquemos que las imágenes se han procesado correctamente.

In [ ]:
# Display a grid of processed sample images
plt.figure(figsize=(12, 6))
for i in range(15):  # Display 15 sample images
    plt.subplot(3, 5, i+1)
    plt.imshow(processed_images[i])
    plt.axis('off')
plt.tight_layout()
plt.show()

4. **Crear DataLoader**:  
   - Usar batches de 64-128 imágenes.  
   - *pista*: `tf.data.Dataset.from_tensor_slices` permite crear un pipeline eficiente.  

In [ ]:
import tensorflow as tf

# Set batch size
batch_size = 64

# Split the data into training and validation sets (90% training, 10% validation)
train_size = int(0.9 * len(processed_images))
train_images = processed_images[:train_size]
val_images = processed_images[train_size:]

print(f"Training images: {len(train_images)}, Validation images: {len(val_images)}")

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(train_images)
val_dataset = tf.data.Dataset.from_tensor_slices(val_images)

# Configure the datasets for performance
train_dataset = train_dataset.shuffle(buffer_size=1000)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

# Verify the dataset shapes
for x_batch in train_dataset.take(1):
    print(f"Input batch shape: {x_batch.shape}")
    # Verify that the data is normalized properly
    print(f"Data range: [{tf.reduce_min(x_batch).numpy()}, {tf.reduce_max(x_batch).numpy()}]")

print(f"Training batches: {tf.data.experimental.cardinality(train_dataset).numpy()}")
print(f"Validation batches: {tf.data.experimental.cardinality(val_dataset).numpy()}")

---

### **2. Diseño del VAE Convolucional**  
**Objetivo**: Construir un VAE con encoder y decoder convolucionales.  

#### **Encoder**:  
- **Capas**:  
  - 4 capas convolucionales (`Conv2D`) con activación `LeakyReLU` o `ReLU`.  
  - Reducir dimensiones espaciales progresivamente (ej: 64x64 → 32x32 → 16x16).  
- **Capa Latente**:  
  - Dos salidas: `z_mean` y `z_log_var` (media y log-varianza de la distribución latente).  
  - Dimensión del espacio latente: 256.  
- *pista*: Usar `Flatten` antes de las capas densas para `z_mean` y `z_log_var`.  

#### **Reparameterization Trick**:  
- Muestrear `z` usando:  
  ```  
  z = z_mean + exp(z_log_var * 0.5) * epsilon  
  ```  
  donde `epsilon ~ N(0, 1)`.  

#### **Decoder**:  
- **Capas**:  
  - 4 capas `Conv2DTranspose` o `UpSampling2D + Conv2D` para aumentar resolución.  
  - Usar activación `sigmoid` en la última capa si las imágenes están en `[0, 1]`.  
- *pista*: Asegurar que la salida final tenga las mismas dimensiones que la entrada (64x64x3).  

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Set hyperparameters
latent_dim = 256  # Size of latent space
input_shape = (64, 64, 3)  # Shape of input images
dropout_rate = 0.2  # Dropout rate for regularization

# Create the encoder
def build_encoder(input_shape, latent_dim, dropout_rate=0.2):
    encoder_inputs = layers.Input(shape=input_shape)
    
    # Convolutional layers with LeakyReLU activation and dropout for regularization
    x = layers.Conv2D(32, 4, strides=2, padding='same')(encoder_inputs)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(dropout_rate)(x)  # Add dropout layer
    # 32x32x32
    
    x = layers.Conv2D(64, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(dropout_rate)(x)  # Add dropout layer
    # 16x16x64
    
    x = layers.Conv2D(128, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(dropout_rate)(x)  # Add dropout layer
    # 8x8x128
    
    x = layers.Conv2D(256, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(dropout_rate)(x)  # Add dropout layer
    # 4x4x256
    
    # Flatten the features
    x = layers.Flatten()(x)
    
    # Output layers for mean and log variance
    z_mean = layers.Dense(latent_dim, name='z_mean')(x)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
    
    # Create the encoder model
    encoder = Model(encoder_inputs, [z_mean, z_log_var], name='encoder')
    return encoder

# Reparameterization trick as a Keras layer
class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch_size = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch_size, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Create the decoder
def build_decoder(latent_dim, output_shape, dropout_rate=0.2):
    latent_inputs = layers.Input(shape=(latent_dim,))
    
    # Calculate dimensions for reshaping
    # For input images of 64x64, after 4 layers of Conv2D with stride 2, we get 4x4
    units = 4 * 4 * 256  # Width * Height * Channels
    
    # Dense layer and reshape
    x = layers.Dense(units)(latent_inputs)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(dropout_rate)(x)  # Add dropout layer
    x = layers.Reshape((4, 4, 256))(x)
    # 4x4x256
    
    # Transposed convolution layers to increase resolution
    x = layers.Conv2DTranspose(128, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(dropout_rate)(x)  # Add dropout layer
    # 8x8x128
    
    x = layers.Conv2DTranspose(64, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(dropout_rate)(x)  # Add dropout layer
    # 16x16x64
    
    x = layers.Conv2DTranspose(32, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.Dropout(dropout_rate)(x)  # Add dropout layer
    # 32x32x32
    
    # Final layer with sigmoid activation for output in range [0, 1]
    # No dropout in the final output layer
    decoder_outputs = layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='sigmoid')(x)
    # 64x64x3
    
    # Create the decoder model
    decoder = Model(latent_inputs, decoder_outputs, name='decoder')
    return decoder

# Build the encoder and decoder with dropout regularization
encoder = build_encoder(input_shape, latent_dim, dropout_rate)
encoder.summary()

# Build the decoder with dropout regularization
decoder = build_decoder(latent_dim, input_shape, dropout_rate)
decoder.summary()

# Define the VAE model
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.sampling = Sampling()
        self.total_loss_tracker = tf.keras.metrics.Mean(name='total_loss')
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name='reconstruction_loss')
        self.kl_loss_tracker = tf.keras.metrics.Mean(name='kl_loss')
        # Add validation metrics
        self.val_total_loss_tracker = tf.keras.metrics.Mean(name='val_loss')
        self.val_reconstruction_loss_tracker = tf.keras.metrics.Mean(name='val_reconstruction_loss')
        self.val_kl_loss_tracker = tf.keras.metrics.Mean(name='val_kl_loss')
    
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.val_total_loss_tracker,
            self.val_reconstruction_loss_tracker,
            self.val_kl_loss_tracker
        ]
    
    def call(self, inputs):
        z_mean, z_log_var = self.encoder(inputs)
        z = self.sampling([z_mean, z_log_var])
        reconstructed = self.decoder(z)
        return reconstructed
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            # Get encoder outputs
            z_mean, z_log_var = self.encoder(data)
            # Sample from the latent distribution
            z = self.sampling([z_mean, z_log_var])
            # Decode the sampled vector
            reconstructed = self.decoder(z)
            
            # Calculate reconstruction loss (pixel-wise binary crossentropy)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstructed),
                    axis=(1, 2)
                )
            )
            
            # Calculate KL divergence loss
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            
            # Total loss is the sum of reconstruction loss and KL loss
            total_loss = reconstruction_loss + kl_loss
        
        # Get gradients and apply them
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        # Update metrics
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        # Return metrics dictionary
        return {
            'loss': self.total_loss_tracker.result(),
            'reconstruction_loss': self.reconstruction_loss_tracker.result(),
            'kl_loss': self.kl_loss_tracker.result()
        }
    
    def test_step(self, data):
        # Get encoder outputs
        z_mean, z_log_var = self.encoder(data)
        # Sample from the latent distribution
        z = self.sampling([z_mean, z_log_var])
        # Decode the sampled vector
        reconstructed = self.decoder(z)
        
        # Calculate reconstruction loss (pixel-wise binary crossentropy)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                tf.keras.losses.binary_crossentropy(data, reconstructed),
                axis=(1, 2)
            )
        )
        
        # Calculate KL divergence loss
        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
        )
        
        # Total loss is the sum of reconstruction loss and KL loss
        total_loss = reconstruction_loss + kl_loss
        
        # Update validation metrics
        self.val_total_loss_tracker.update_state(total_loss)
        self.val_reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.val_kl_loss_tracker.update_state(kl_loss)
        
        # Return metrics dictionary
        return {
            'loss': self.val_total_loss_tracker.result(),
            'reconstruction_loss': self.val_reconstruction_loss_tracker.result(),
            'kl_loss': self.val_kl_loss_tracker.result()
        }

# Create the VAE model
vae = VAE(encoder, decoder)

# Compile the model
# Need to provide a dummy loss even though we implement our own in train_step
vae.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    # Adding a dummy loss function to prevent the ValueError
    loss=None  # Set to None since we have custom train_step and test_step methods
)

# Display the model architecture summary
print("VAE Architecture with Dropout Created Successfully")

---

### **3. Función de Pérdida**  
**Objetivo**: Definir la pérdida del VAE (reconstrucción + divergencia KL).  
1. **Reconstrucción**:  
   - Error cuadrático medio (MSE) entre imágenes originales y reconstruidas.  
   - *Alternativa*: Pérdida de entropía cruzada binaria (BCE) si se usa `sigmoid`.  
2. **Divergencia KL**:  
   - Calculada entre la distribución latente y una normal estándar:  
     ```  
     KL = -0.5 * sum(1 + z_log_var - z_mean^2 - exp(z_log_var))  
     ```  
3. **Pérdida Total**: `loss = reconstruction_loss + beta * KL_loss` (beta=1 por defecto).  
   - *pista*: Usar `beta` como hiperparámetro para ajustar el trade-off.

In [ ]:
# Note: We've already implemented the loss functions inside our VAE class in the previous step
# Let's experiment with different beta values for the KL divergence term

# Define a custom VAE with beta parameter
class BetaVAE(Model):
    def __init__(self, encoder, decoder, beta=1.0, **kwargs):
        super(BetaVAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.sampling = Sampling()
        self.beta = beta
        self.total_loss_tracker = tf.keras.metrics.Mean(name='total_loss')
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name='reconstruction_loss')
        self.kl_loss_tracker = tf.keras.metrics.Mean(name='kl_loss')
        # Add validation metrics
        self.val_total_loss_tracker = tf.keras.metrics.Mean(name='val_loss')
        self.val_reconstruction_loss_tracker = tf.keras.metrics.Mean(name='val_reconstruction_loss')
        self.val_kl_loss_tracker = tf.keras.metrics.Mean(name='val_kl_loss')
    
    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
            self.val_total_loss_tracker,
            self.val_reconstruction_loss_tracker,
            self.val_kl_loss_tracker
        ]
    
    def call(self, inputs):
        z_mean, z_log_var = self.encoder(inputs)
        z = self.sampling([z_mean, z_log_var])
        reconstructed = self.decoder(z)
        return reconstructed
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            # Get encoder outputs
            z_mean, z_log_var = self.encoder(data)
            # Sample from the latent distribution
            z = self.sampling([z_mean, z_log_var])
            # Decode the sampled vector
            reconstructed = self.decoder(z)
            
            # Calculate reconstruction loss
            # Option 1: Mean Squared Error (MSE)
            # reconstruction_loss = tf.reduce_mean(
            #     tf.reduce_sum(
            #         tf.square(data - reconstructed), 
            #         axis=(1, 2)
            #     )
            # )
            
            # Option 2: Binary Cross-Entropy (BCE) - better for image pixels in [0,1]
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstructed),
                    axis=(1, 2)
                )
            )
            
            # Calculate KL divergence loss
            # KL divergence between the latent distribution and a standard normal
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            
            # Total loss with beta parameter to control the trade-off
            total_loss = reconstruction_loss + self.beta * kl_loss
        
        # Get gradients and apply them
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        # Update metrics
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        
        # Return metrics dictionary
        return {
            'loss': self.total_loss_tracker.result(),
            'reconstruction_loss': self.reconstruction_loss_tracker.result(),
            'kl_loss': self.kl_loss_tracker.result()
        }
    
    def test_step(self, data):
        # Get encoder outputs
        z_mean, z_log_var = self.encoder(data)
        # Sample from the latent distribution
        z = self.sampling([z_mean, z_log_var])
        # Decode the sampled vector
        reconstructed = self.decoder(z)
        
        # Calculate reconstruction loss (pixel-wise binary crossentropy)
        reconstruction_loss = tf.reduce_mean(
            tf.reduce_sum(
                tf.keras.losses.binary_crossentropy(data, reconstructed),
                axis=(1, 2)
            )
        )
        
        # Calculate KL divergence loss
        kl_loss = -0.5 * tf.reduce_mean(
            tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
        )
        
        # Total loss with beta parameter
        total_loss = reconstruction_loss + self.beta * kl_loss
        
        # Update validation metrics
        self.val_total_loss_tracker.update_state(total_loss)
        self.val_reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.val_kl_loss_tracker.update_state(kl_loss)
        
        # Return metrics dictionary
        return {
            'loss': self.val_total_loss_tracker.result(),
            'reconstruction_loss': self.val_reconstruction_loss_tracker.result(),
            'kl_loss': self.val_kl_loss_tracker.result()
        }

# Create models with different beta values to experiment with
# Higher beta enforces more regularization to the latent space
# Lower beta allows more focus on reconstruction quality
beta_values = [0.5, 1.0, 2.0]
beta_vae_models = {}

for beta in beta_values:
    beta_vae = BetaVAE(encoder, decoder, beta=beta)
    beta_vae.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss=None  # Set to None since we have custom methods
    )
    beta_vae_models[f"beta_{beta}"] = beta_vae

print("Created VAE models with different beta values:")
for beta_name, model in beta_vae_models.items():
    print(f"- {beta_name}")

# Visualization of the loss function components
def plot_loss_components():
    plt.figure(figsize=(10, 6))
    
    # Create sample data for visualization
    z_mean_range = np.linspace(-3, 3, 100)
    z_log_var_range = np.linspace(-3, 3, 100)
    
    kl_losses = []
    for z_mean in z_mean_range:
        for z_log_var in z_log_var_range:
            # KL divergence term for a single latent dimension
            kl = -0.5 * (1 + z_log_var - z_mean**2 - np.exp(z_log_var))
            kl_losses.append((z_mean, z_log_var, kl))
    
    # Plot KL divergence as a function of z_mean and z_log_var
    z_means = [x[0] for x in kl_losses]
    z_log_vars = [x[1] for x in kl_losses]
    kl_values = [x[2] for x in kl_losses]
    
    plt.scatter(z_means, z_log_vars, c=kl_values, cmap='viridis', alpha=0.7)
    plt.colorbar(label='KL Divergence')
    plt.xlabel('z_mean')
    plt.ylabel('z_log_var')
    plt.title('KL Divergence as a function of z_mean and z_log_var')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Illustrate the effect of beta on total loss
    plt.figure(figsize=(10, 6))
    
    # Example fixed values
    recon_loss = 1.0
    kl_loss_values = np.linspace(0, 2, 100)
    
    for beta in beta_values:
        total_loss = recon_loss + beta * kl_loss_values
        plt.plot(kl_loss_values, total_loss, label=f'β = {beta}')
    
    plt.xlabel('KL Loss')
    plt.ylabel('Total Loss (with fixed reconstruction loss)')
    plt.title('Effect of β on Total Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Run the visualization
plot_loss_components()

---

### **4. Entrenamiento**  
**Objetivo**: Entrenar el modelo y monitorear la generación.  
**Instrucciones**:  
1. **Compilar el modelo**:  
   - Optimizador: `Adam` con learning rate=0.0005.  
   - Métricas opcionales: Seguir `loss`, `reconstruction_loss`, `KL_loss` por separado.  
2. **Callbacks útiles**:  
   - `ModelCheckpoint`: Guardar el mejor modelo.  
   - `TensorBoard`: Visualizar curvas de pérdida.  
   - `EarlyStopping`: Detener el entrenamiento si la pérdida de validación no mejora.  
   - `ReduceLROnPlateau`: Reducir la tasa de aprendizaje si la pérdida se estanca.  
3. **Entrenar**:  
   - Épocas: 30-50 (CelebA requiere más tiempo que CIFAR-10).  
   - Batch size: 64-128 (depende de la memoria de GPU).  
4. **Monitoreo visual**:  
   - Generar rostros nuevos cada 5 épocas muestreando `z ~ N(0, 1)`.  

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, LambdaCallback, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

# Create directories for saving model checkpoints and generated images
model_dir = os.path.join('model', 'vae_checkpoints')
generated_dir = os.path.join('model', 'generated_faces')
logs_dir = os.path.join('model', 'logs', 'vae_training')

os.makedirs(model_dir, exist_ok=True)
os.makedirs(generated_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

# Create a fixed latent sample for consistent generation during training
fixed_z_sample = tf.random.normal(shape=(15, latent_dim))

# Function to generate and save images during training
def generate_and_save_images(epoch, z_sample=None, n_samples=15, rows=3, cols=5):
    if z_sample is None:
        # Use the fixed sample for consistency
        z_sample = fixed_z_sample
        
    # Generate images from the decoder
    generated_images = decoder(z_sample)
    
    # Plot the generated images
    plt.figure(figsize=(12, 6))
    for i in range(n_samples):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(generated_images[i])
        plt.axis('off')
        
    plt.tight_layout()
    plt.suptitle(f'Generated Faces - Epoch {epoch}')
    
    # Save the plot
    save_path = os.path.join(generated_dir, f'generated_faces_epoch_{epoch}.png')
    plt.savefig(save_path)
    plt.close()
    
    return save_path

# Custom callback class for generating images
class GenerateImagesCallback(tf.keras.callbacks.Callback):
    def __init__(self, z_sample, interval=5):
        super(GenerateImagesCallback, self).__init__()
        self.z_sample = z_sample
        self.interval = interval
        
    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.interval == 0 or epoch == 0:
            generate_and_save_images(epoch + 1, self.z_sample)

# 1. TensorBoard callback for visualizing training progress
tensorboard_callback = TensorBoard(
    log_dir=logs_dir,
    histogram_freq=1,
    write_graph=True,
    update_freq='epoch'  # Only update per epoch to avoid None values
)

# 2. ModelCheckpoint to save the best model during training
checkpoint_callback = ModelCheckpoint(
    filepath=os.path.join(model_dir, 'vae_epoch_{epoch:03d}_loss_{loss:.4f}.weights.h5'),
    save_best_only=True,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# 3. Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Number of epochs with no improvement after which training will stop
    restore_best_weights=True,  # Restore model weights from the epoch with the best value of the monitored quantity
    verbose=1
)

# 4. Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,  # Factor by which the learning rate will be reduced
    patience=5,  # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6,  # Lower bound on the learning rate
    verbose=1
)

# 5. Generate images callback
generate_images_callback = GenerateImagesCallback(fixed_z_sample, interval=5)

# Build the model by passing a sample batch through it
print("Building the VAE model...")
# Get a sample batch from the dataset
for x_batch in train_dataset.take(1):
    # Run a forward pass to build the model
    _ = vae(x_batch)
    print(f"Model successfully built with input shape: {x_batch.shape}")

# Generate initial images before training
print("Generating images before training...")
initial_images_path = generate_and_save_images(0, fixed_z_sample)
print(f"Initial images saved to: {initial_images_path}")

# Train the model with callbacks - increase epochs but with early stopping to prevent overfitting
history = vae.fit(
    train_dataset,
    epochs=100,  # Increased from 50 to 100, early stopping will prevent overfitting
    validation_data=val_dataset,
    callbacks=[
        checkpoint_callback,
        tensorboard_callback,
        generate_images_callback,
        early_stopping,
        reduce_lr
    ]
)

# Print training summary
print(f"Training completed for {len(history.history['loss'])} epochs")
print(f"Best validation loss: {min(history.history['val_loss'])}")

# Save the final model weights
final_model_path = os.path.join(model_dir, 'vae_final_model.weights.h5')
vae.save_weights(final_model_path)
print(f"Final model saved to {final_model_path}")

# Explore the latent space by interpolating between two random points
n_interpolations = 10  # Number of interpolation steps
z_start = tf.random.normal(shape=(1, latent_dim))  # Random point in latent space
z_end = tf.random.normal(shape=(1, latent_dim))  # Another random point

# Linear interpolation in latent space
z_interpolated = tf.concat([
    z_start + (z_end - z_start) * t / n_interpolations
    for t in range(n_interpolations + 1)
], axis=0)

# Generate and save interpolated images
interpolation_path = generate_and_save_images(
    epoch='interp',
    z_sample=z_interpolated,
    n_samples=n_interpolations + 1,
    rows=2,
    cols=6
)
print(f"Interpolation images saved to: {interpolation_path}")

# Visualize the training history (making sure we handle the case if validation metrics are missing)
plt.figure(figsize=(15, 5))

# Plot training metrics
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Train')
if 'val_loss' in history.history:
    plt.plot(history.history['val_loss'], label='Validation')
plt.title('Total Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(history.history['reconstruction_loss'], label='Train')
if 'val_reconstruction_loss' in history.history:
    plt.plot(history.history['val_reconstruction_loss'], label='Validation')
plt.title('Reconstruction Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(history.history['kl_loss'], label='Train')
if 'val_kl_loss' in history.history:
    plt.plot(history.history['val_kl_loss'], label='Validation')
plt.title('KL Divergence Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

---

### **5. Generación y Evaluación**  
**Objetivo**: Crear rostros nuevos y evaluar la calidad.  
**Instrucciones**:  
1. **Generación**:  
   - Muestrear vectores `z` de `N(0, I)` y pasarlos por el decoder.  
   - *pista*: Si el espacio latente es 2D, se puede visualizar una cuadrícula de rostros variando `z`.  
2. **Evaluación cualitativa**:  
   - ¿Los rostros generados son diversos y realistas?  
   - ¿Existe "mode collapse" (todas las generaciones son similares)?  
3. **Evaluación cuantitativa (opcional)**:  
   - Métricas como **FID** (Fréchet Inception Distance) comparan distribuciones de imágenes reales y generadas.  